# Generator 🗂
### The generator is accessible through the `generate` command (abbreviated `g`).

In [2]:
! D generate --help

Usage: D generate [OPTIONS] COMMAND [ARGS]...

  Adds models, routes, and other resources

Options:
  --dry      Display output without creating files.
  --force    Override any conflicting files.
  --verbose  Run in verbose mode.
  --help     Show this message and exit.

Commands:
  admin       Generates an admin model within the admin package.
  fixture     Generates model fixtures.
  form        Generates a model form within the forms package.
  manager     Generates a model manager under the model managers directory.
  model       Generates a model under the models directory.
  resource    Generates an app resource.
  serializer  Generates a serializer for a given model.
  template    Generates an html template.
  test        Generates a new TestCase.
  view        Generates a view function or class.
  viewset     Generates a viewset for a serializable model.


## Generating Models

In [120]:
! D generate model --help

Usage: D generate model [OPTIONS] NAME [FIELDS]...

  Generates a model under the models directory. One can specify multiple
  attributes after the model's name, like so:

      D g model track int:number char:title fk:album bool:is_favorite

  This will generate a Track model and add a foreign key of Album. If the
  model is to be added to admin.site one can optionally opt in by specifying
  the --register-admin flag.

Options:
  -a, --abstract                  Creates an abstract model type.
  -t, --test-case                 Creates a TestCase for model.
  -f, --full                      Adds all related resources and TestCase
  --register-admin                Register model to admin site.
  --register-inline               Register model to admin site as inline.
  -m, --is-managed                Add created_by and updated_by fields.
  -i, --inherits, --extends TEXT  Add model inheritance.
  --app TEXT                      If base model inherits is in another app.
  --api             

In order to generate a model, specify the type identifier and then the name of the attribute field. Type identifiers are abbreviated to a more generic name that omits the word `Field`. The input here is case-insensitive, but the fields will be properly CamelCased in the corresponding Python file as in the example below:

In [68]:
! D generate model album text:title image:artwork bool:is_compilation

Successfully created album.py for Album.


This would add the following model `album.py` under the `models` directory within the music app:

In [140]:
! cat models/album.py

import uuid
from django.db import models
from django.urls import reverse
from django.template.defaultfilters import slugify
from django.utils.translation import gettext_lazy as _


class Album(models.Model):
    
    # Default fields. Used for record-keeping.
    uuid = models.UUIDField(default=uuid.uuid4, editable=False)
    slug = models.SlugField(_('slug'), max_length=250, unique=True, editable=False, blank=True)
    created_at = models.DateTimeField(_('created at'), auto_now_add=True, editable=False)
    updated_at = models.DateTimeField(_('updated at'), auto_now=True, editable=False)

    class Meta:
        
        db_table = 'music_albums'
        indexes = [models.Index(fields=['created_at'])]
        ordering = ['-created_at']

    def save(self, *args, **kwargs):
        # Generate a Medium-like URL slugs:
        # slugify(f'{__SomeCharField__}{str(self.uuid)[-12:]}')
        self.slug = slugify(f'{str(self.uuid)[-12:]}')
        super().save(*args, **kwargs)

    def __str

Note the presence of the `--inherits` flag. You can specify a base model and the generated model will extend it. For example (from within the music directory):

In [144]:
! D generate model ep --inherits album

Successfully created ep.py for Ep.


In [145]:
! cat models/ep.py

import uuid
from django.db import models
from django.urls import reverse
from django.template.defaultfilters import slugify
from django.utils.translation import gettext_lazy as _
from .album import Album


class Ep(Album):
    
    # Default fields. Used for record-keeping.
    uuid = models.UUIDField(default=uuid.uuid4, editable=False)
    slug = models.SlugField(_('slug'), max_length=250, unique=True, editable=False, blank=True)
    created_at = models.DateTimeField(_('created at'), auto_now_add=True, editable=False)
    updated_at = models.DateTimeField(_('updated at'), auto_now=True, editable=False)

    class Meta:
        
        db_table = 'music_eps'
        indexes = [models.Index(fields=['created_at'])]
        ordering = ['-created_at']

    def save(self, *args, **kwargs):
        # Generate a Medium-like URL slugs:
        # slugify(f'{__SomeCharField__}{str(self.uuid)[-12:]}')
        self.slug = slugify(f'{str(self.uuid)[-12:]}')


    def __str__(self):
        return 

**Defaults**

As one can see, `class Meta` and `_str_` are added to a model by default along with `uuid`, `slug`, `created_at` and `updated_at` fields.
The `db_table` name is inferred from the name of the app and the current model while the ordering attribute is defined based on the default `created_at` field.

**Relationships**

If a relationship identifier is passed, the attribute name will be used as the name of the model it relates to.
Specifying a relationship also checks the current app scope for the specified related model. If such model does not exist in scope, the CLI will prompt you to create the missing model. How to invoke the command:

In [156]:
! D generate --dry model track char:title

Filename: __init__.py
Filepath: /Users/lneto/Open-source/django-clite/docs/website/website/music/models/

---- Begin content ----
from .track import Track
---- End content ----


Filename: track.py
Filepath: /Users/lneto/Open-source/django-clite/docs/website/website/music/models/

---- Begin content ----
import uuid
from django.db import models
from django.urls import reverse
from django.template.defaultfilters import slugify
from django.utils.translation import gettext_lazy as _


class Track(models.Model):
    
    # Default fields. Used for record-keeping.
    uuid = models.UUIDField(default=uuid.uuid4, editable=False)
    slug = models.SlugField(_('slug'), max_length=250, unique=True, editable=False, blank=True)
    created_at = models.DateTimeField(_('created at'), auto_now_add=True, editable=False)
    updated_at = models.DateTimeField(_('updated at'), auto_now=True, editable=False)

    class Meta:
        
        db_table = 'music_tracks'
        indexes = [models.Index(fields